

### Generación de Modelos Predictivos

Este notebook se encarga de generar modelos predictivos.  Se puede correr en Google Colab o en el iMac.

Para usarlo en Google Colab, usar tipo_a_usar='GPU'
Para usarlo en el imac, usar tipo_a_usar='CPU'



In [17]:
tipo_a_usar='CPU'

In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score,roc_auc_score, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb

In [19]:
if tipo_a_usar =='GPU':
  from google.colab import drive
  drive.mount('/content/drive')

Los datos de origen es el mismo sea desde Google Colab o desde el iMac.  La diferencia es que el primero usar referencia a Google Drive y el otro acceso directo desde el sistema operativo

In [20]:
if tipo_a_usar =='GPU':
  DATOS_LIMPIOS = pd.read_pickle('/content/drive/MyDrive//Innovaciones Tecnológicas Aplicadas/Universidad Autónoma de Occidente/GoogleColab/DATOS3_35.pkl')

if tipo_a_usar == 'CPU':
  DATOS_LIMPIOS = pd.read_pickle('/Users/jaimereinoso/DESARROLLO/MCD/proyectodegrado2/checkPoints/DATOS_LIMPIOS.pkl')


In [21]:
# LABEL viene con 3 valores. 0:  los que ganaron.  
# 1:  los que perdieron o cancelaron, 
# -1: los que no sabemos pues aún están viendo las asignaturas 
#     o no se nos informó su resultado.  Estos ultimos son para hacer predicciones

DATOS_LIMPIOS['LABEL'].value_counts()

LABEL
 0    4820
 1    1791
-1     534
Name: count, dtype: int64

In [23]:
DATOS_LIMPIOS.shape

(7145, 1955)

In [5]:
#!pip install imbalanced-learn
from imblearn.over_sampling import SMOTE

# solo tomamos en cuenta los 0 (no desertores) y 1 (desertores)
data = DATOS_LIMPIOS[DATOS_LIMPIOS['LABEL']>=0].copy() 

X = data.drop('LABEL', axis=1)
y = data['LABEL']

# procedemos a hacer el balanceo
smote = SMOTE(sampling_strategy='auto')

# Aplica SMOTE para generar muestras sintéticas
X_resampled, y_resampled = smote.fit_resample(X, y)

In [6]:
print(X_resampled.shape)
y_resampled.value_counts()

(9640, 1954)


LABEL
0    4820
1    4820
Name: count, dtype: int64

In [17]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2)
X_train.shape

(7712, 1954)

In [18]:

# Definir una lista de algoritmos que deseas probar, y si deben usar CPU o GPU
# si usan CPU se pueden correr en el iMac.  Si usan GPU se corren en GOOGLE COLAB 

algorithms = [
#    ('RandomForest',
#         RandomForestClassifier(), {
#             'n_estimators': [50, 100, 200,400],
#             'max_depth': [None, 10, 20, 30],
#             'min_samples_split': [2, 5, 10]
#         },
#         'CPU'
#     ),
    ('DecisionTree',
        DecisionTreeClassifier(), {
            'criterion': ['gini', 'entropy'],
            'max_depth': [None, 10, 20, 30, 40, 50],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4]
        },
        'CPU'
    ),
    # ('SVM', SVC(), {
    #     'C': [0.1],
    #     'kernel': ['linear']
    # },
    # 'CPU'),
    ('XGBoost', xgb.XGBClassifier(), {
        'n_estimators': [50, 100, 200],
        'learning_rate': [0.01, 0.1, 0.2],
        'max_depth': [3, 4, 5],
        'tree_method':['hist']  # no aguantó gpu_hist en google_colab
    },
    'GPU')
]

In [19]:
# solo usar si se está en Google Colab para confirmar la asignación de una
# tarjeta gráfica

if tipo_a_usar == 'GPU':
  gpu_info = !nvidia-smi
  gpu_info = '\n'.join(gpu_info)
  if gpu_info.find('failed') >= 0:
    print('Not connected to a GPU')
  else:
    print(gpu_info)

In [20]:
print(X_train.shape)
print(X_test.shape)

(7712, 1954)
(1928, 1954)


In [21]:
import joblib
import datetime


for algo_name, model, param_grid, tipo in algorithms:

  if tipo == tipo_a_usar:

    modelo_a_salvar = []

    print('algo:', algo_name, 'model:',model, 'param grid:', param_grid)
    grid_search = GridSearchCV(model, param_grid, cv=5, scoring='roc_auc')
    grid_search.fit(X_train, y_train)

    # Evaluar el modelo
    y_pred = grid_search.best_estimator_.predict(X_test)

    # vamos a salvarlo en modelo_a_salvar
    modelo_a_salvar.append(algo_name)

    accuracy = accuracy_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred)
    conf_matrix = confusion_matrix(y_test, y_pred)

    modelo_a_salvar.append(accuracy)
    modelo_a_salvar.append(roc_auc)
    modelo_a_salvar.append(conf_matrix)
    modelo_a_salvar.append(grid_search.best_estimator_)
    modelo_a_salvar.append(grid_search.best_params_)

    print("Algoritmo:", algo_name)
    print("Accuracy:", accuracy)
    print("Roc_Auc:", roc_auc)
    print("Conf_Matrix:", conf_matrix)

    # Obtén la fecha y hora actual
    now = datetime.datetime.now()
    nombre_archivo = algo_name + now.strftime("%Y-%m-%d_%H-%M-%S") + ".pkl"

    if tipo_a_usar =='GPU':
      from google.colab import files
      joblib.dump(modelo_a_salvar,nombre_archivo)
      files.download(algo_name+'.pkl')
      # queda en download del iMac

    else:
      joblib.dump(modelo_a_salvar,nombre_archivo)
      print('entro aqui')


algo: DecisionTree model: DecisionTreeClassifier() param grid: {'criterion': ['gini', 'entropy'], 'max_depth': [None, 10, 20, 30, 40, 50], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4]}
Algoritmo: DecisionTree
Accuracy: 0.8905601659751037
Roc_Auc: 0.8905139573424015
Conf_Matrix: [[871  99]
 [112 846]]
entro aqui


Para regresion logistica toca primero eliminar variables altamente correlacionadas



In [6]:
dataLR = DATOS_LIMPIOS.copy()
correlation_matrix = dataLR.corr()

# Crear una máscara booleana para las variables altamente correlacionadas
correlation_threshold = 0.7  # Define el umbral de correlación deseado
highly_correlated = (correlation_matrix.abs() >= correlation_threshold) & (correlation_matrix.abs() < 1.0)

# Identificar las variables altamente correlacionadas sin quitar LABEL
correlated_columns = set()
for col in highly_correlated.columns:
    if col!='LABEL':
        correlated_columns.update(highly_correlated.index[highly_correlated[col]])

# Eliminar las variables altamente correlacionadas del DataFrame
dataLR = dataLR.drop(columns=correlated_columns)

In [9]:
print(DATOS_LIMPIOS.shape)
dataLR.shape


(7145, 1955)


(7145, 1119)

In [7]:
#!pip install imbalanced-learn
from imblearn.over_sampling import SMOTE

# solo tomamos en cuenta los 0 (no desertores) y 1 (desertores)
data = dataLR[dataLR['LABEL']>=0].copy() 

X = data.drop('LABEL', axis=1)
y = data['LABEL']

# procedemos a hacer el balanceo
smote = SMOTE(sampling_strategy='auto')

# Aplica SMOTE para generar muestras sintéticas
X_resampled, y_resampled = smote.fit_resample(X, y)

In [14]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2)
X_train.shape

(7712, 1118)

In [15]:
# Definir una lista de algoritmos que deseas probar, y si deben usar CPU o GPU
# si usan CPU se pueden correr en el iMac.  Si usan GPU se corren en GOOGLE COLAB 

algorithms = [
#    ('RandomForest',
#         RandomForestClassifier(), {
#             'n_estimators': [50, 100, 200,400],
#             'max_depth': [None, 10, 20, 30],
#             'min_samples_split': [2, 5, 10]
#         },
#         'CPU'
#     ),
    # ('DecisionTree',
    #     DecisionTreeClassifier(), {
    #         'criterion': ['gini', 'entropy'],
    #         'max_depth': [None, 10, 20, 30, 40, 50],
    #         'min_samples_split': [2, 5, 10],
    #         'min_samples_leaf': [1, 2, 4]
    #     },
    #     'CPU'
    # ),
        ('LogisticRegression',
        LogisticRegression(), {
            'penalty': ['l1', 'l2'],           # Tipo de regularización
            'C': [0.001, 0.01, 0.1, 1, 10],   # Parámetro de regularización (inverso de la fuerza de regularización)
            'solver': ['liblinear', 'saga']
        },
        'CPU'
    ),
    # ('SVM', SVC(), {
    #     'C': [0.1],
    #     'kernel': ['linear']
    # },
    # 'CPU'),
    ('XGBoost', xgb.XGBClassifier(), {
        'n_estimators': [50, 100, 200],
        'learning_rate': [0.01, 0.1, 0.2],
        'max_depth': [3, 4, 5],
        'tree_method':['hist']  # no aguantó gpu_hist en google_colab
        },
        'GPU'
    )
]

In [16]:
import joblib
import datetime


for algo_name, model, param_grid, tipo in algorithms:

  if tipo == tipo_a_usar:

    modelo_a_salvar = []

    print('algo:', algo_name, 'model:',model, 'param grid:', param_grid)
    grid_search = GridSearchCV(model, param_grid, cv=5, scoring='roc_auc')
    grid_search.fit(X_train, y_train)

    # Evaluar el modelo
    y_pred = grid_search.best_estimator_.predict(X_test)

    # vamos a salvarlo en modelo_a_salvar
    modelo_a_salvar.append(algo_name)

    accuracy = accuracy_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred)
    conf_matrix = confusion_matrix(y_test, y_pred)

    modelo_a_salvar.append(accuracy)
    modelo_a_salvar.append(roc_auc)
    modelo_a_salvar.append(conf_matrix)
    modelo_a_salvar.append(grid_search.best_estimator_)
    modelo_a_salvar.append(grid_search.best_params_)

    print("Algoritmo:", algo_name)
    print("Accuracy:", accuracy)
    print("Roc_Auc:", roc_auc)
    print("Conf_Matrix:", conf_matrix)

    # Obtén la fecha y hora actual
    now = datetime.datetime.now()
    nombre_archivo = algo_name + now.strftime("%Y-%m-%d_%H-%M-%S") + ".pkl"

    if tipo_a_usar =='GPU':
      from google.colab import files
      joblib.dump(modelo_a_salvar,nombre_archivo)
      files.download(algo_name+'.pkl')
      # queda en download del iMac

    else:
      joblib.dump(modelo_a_salvar,nombre_archivo)
      print('entro aqui')

algo: LogisticRegression model: LogisticRegression() param grid: {'penalty': ['l1', 'l2'], 'C': [0.001, 0.01, 0.1, 1, 10], 'solver': ['liblinear', 'saga']}


/Users/jaimereinoso/DESARROLLO/MCD/proyectodegrado2/venv/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/jaimereinoso/DESARROLLO/MCD/proyectodegrado2/venv/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/jaimereinoso/DESARROLLO/MCD/proyectodegrado2/venv/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/jaimereinoso/DESARROLLO/MCD/proyectodegrado2/venv/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/jaimereinoso/DESARROLLO/MCD/proyectodegrado2/venv/lib/python3.11/site-packages/sklearn/linear_model/_sag.py:3

Algoritmo: LogisticRegression
Accuracy: 0.8661825726141079
Roc_Auc: 0.8660186387100393
Conf_Matrix: [[875 131]
 [127 795]]
entro aqui
